In [ ]:
import pathlib
from itertools import count

import gif
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
from sklearn.linear_model import ElasticNet, ElasticNetCV, LinearRegression
from sklearn.metrics import mean_absolute_error, mean_squared_error
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import MinMaxScaler

In [ ]:
# to save time, set plot_figs to false
PLOT_FIGS = True

# setting plotting params
%matplotlib inline

fig_height = 7.0
fig_width = fig_height * 1.618
plt.rcParams["figure.figsize"] = [fig_width, fig_height]
plt.rcParams["figure.dpi"] = 300

sns.set_context("talk")
sns.set_style("darkgrid")

# start the figure counter
fig_counter = count(start=1)

In [ ]:
# creating folder for figures
p = pathlib.Path("01_linear_forecast", "dummy.txt")
if not p.parent.exists():
    p.parent.mkdir()

In [ ]:
df_in = pd.read_csv(
    "https://raw.githubusercontent.com/facebook/prophet/main/examples/example_wp_log_peyton_manning.csv"
)

df_in = df_in.assign(ds=pd.to_datetime(df_in["ds"]))

In [ ]:
df_in.sample(n=5)

In [ ]:
plt.plot_date(
    x=df_in["ds"],
    y=df_in["y"],
    label="input timeseries",
    fmt="-",
)
plt.legend(loc="upper right")
plt.xlabel("date")
plt.ylabel("visits log transformed ")
plt.title("daily visits Peyton Manning wiki (log)")
if PLOT_FIGS:
    plt.savefig(f"01_linear_forecast/{next(fig_counter)}_overview.jpg")

In [ ]:
# scoping the years 2012,2013,2014
df_train = df_in[(df_in["ds"] > "2012") & (df_in["ds"] < "2015")]
df_test = df_in[(df_in["ds"] > "2015")]

In [ ]:
plt.plot_date(
    x=df_train["ds"],
    y=df_train["y"],
    label="train",
    fmt="-",
)
plt.plot_date(
    x=df_test["ds"],
    y=df_test["y"],
    label="test",
    fmt="-",
)
plt.legend(loc="upper right")
plt.xlabel("date")
plt.ylabel("visits log transformed ")
plt.title("daily visits scoped")
if PLOT_FIGS:
    plt.savefig(f"01_linear_forecast/{next(fig_counter)}_scoped.jpg")

In [ ]:
X_train = df_train["ds"].astype(int).values.reshape(-1, 1)
y_train = df_train["y"].values

X_test = df_test["ds"].astype(int).values.reshape(-1, 1)
y_test = df_test["y"].values

In [ ]:
# creating the model
linear = LinearRegression()

# fit the model
linear.fit(X=X_train, y=y_train)


# inference
y_pred = linear.predict(X=X_test)

In [ ]:
mse = mean_squared_error(y_test, y_pred)
mae = mean_absolute_error(y_test, y_pred)
print(f"mean squared error = {mse:.3}")
print(f"mean absolute error = {mae:.3}")

In [ ]:
plt.plot_date(
    x=df_train["ds"],
    y=df_train["y"],
    label="train",
    fmt="-",
)
plt.plot_date(
    x=df_test["ds"],
    y=df_test["y"],
    label="test",
    fmt="-",
)

plt.plot_date(
    x=df_test["ds"],
    y=y_pred,
    label="prediction",
    fmt="-",
)


plt.legend(loc="upper right")
plt.xlabel("date")
plt.ylabel("visits log transformed ")
plt.title(f"linear regression applied (mse= {mse:.3}, mae={mae:.3})")

if PLOT_FIGS:
    plt.savefig(f"01_linear_forecast/{next(fig_counter)}_linear_predict.jpg")

In [ ]:
# creating dummies for the months
df_dummies = df_in.assign(
    month=df_in["ds"].dt.month.astype("category"), ds_int=df_in["ds"].astype(int)
)

not_dummy = {"y", "ds", "ds_int"}
to_dummy = set(df_dummies.columns) - not_dummy

df_dummies = pd.get_dummies(data=df_dummies, columns=["month"])
all_features = list(set(df_dummies.columns) - {"y", "ds"})

# slicing the input in train test
df_train_dummies = df_dummies[(df_dummies["ds"] > "2012") & (df_dummies["ds"] < "2015")]
df_test_dummies = df_dummies[(df_dummies["ds"] > "2015")]

X_train = df_train_dummies.loc[:, all_features]
y_train = df_train_dummies[["y"]]

X_test = df_test_dummies.loc[:, all_features]
y_test = df_test_dummies[["y"]]

In [ ]:
# create the pipeline
pipeline = make_pipeline(MinMaxScaler(), LinearRegression())

pipeline.fit(X=X_train, y=y_train)

y_pred_dummies = pipeline.predict(X=X_test)

In [ ]:
mse_dummies = mean_squared_error(y_test, y_pred_dummies)
mae_dummies = mean_absolute_error(y_test, y_pred_dummies)
print(f"mean squared error = {mse_dummies:.3}")
print(f"mean absolute error = {mae_dummies:.3}")

In [ ]:
plt.plot_date(
    x=df_train["ds"],
    y=df_train["y"],
    label="train",
    fmt="-",
)
plt.plot_date(
    x=df_test["ds"],
    y=df_test["y"],
    label="test",
    fmt="-",
)

plt.plot_date(
    x=df_test["ds"],
    y=y_pred_dummies,
    label="prediction /w dummies",
    fmt="-",
)


plt.legend(loc="upper right")
plt.xlabel("date")
plt.ylabel("visits log transformed ")
plt.title(f"linear regression applied (mse= {mse_dummies:.3}, mae={mae_dummies:.3})")
if PLOT_FIGS:
    plt.savefig(f"01_linear_forecast/{next(fig_counter)}_dummy_predict.jpg")

In [ ]:
if PLOT_FIGS:
    # define single frame
    @gif.frame
    def create_blend_gif(ratio):
        y_blend = (ratio * y_pred).reshape(-1, 1) + (1 - ratio) * y_pred_dummies
        with sns.axes_style("whitegrid"):
            plt.ylim(y_pred_dummies.min(), y_pred_dummies.max())
            plt.plot_date(
                x=df_test["ds"],
                y=y_blend,
                fmt="-",
            )
            ax = plt.gca()
            ax.get_xaxis().set_visible(False)
            ax.get_yaxis().set_visible(False)

    # loop over all frames
    num_of_frames = 100
    gif_frames = [
        create_blend_gif(i) for i in np.linspace(start=0, stop=1, num=num_of_frames)
    ]

    # freeze on the bounce point
    gif_frames.extend([gif_frames[-1] for i in range(num_of_frames // 4)])

    # add the the original series in reverse
    gif_frames.extend(gif_frames[::-1])

    # export the gif
    gif.save(
        gif_frames,
        "01_linear_forecast/ratio.gif",
        duration=6,
        unit="s",
        between="startend",
        loop=True,
    )

### creating dummies for all 

In [ ]:
# creating dummies for the months
df_all_dummies = df_in.assign(
    ds_int=df_in["ds"].astype(int),
    month=df_in["ds"].dt.month.astype("category"),
    day=df_in["ds"].dt.day.astype("category"),
    weekday=df_in["ds"].dt.weekday.astype("category"),
    weekofyear=df_in["ds"].dt.isocalendar().week.astype("category"),
    is_weekend=df_in["ds"].dt.weekday > 5,
)


not_dummy = {"y", "ds", "ds_int"}
to_dummy = set(df_all_dummies.columns) - not_dummy

df_all_dummies = pd.get_dummies(data=df_all_dummies, columns=list(to_dummy))

all_features = list(set(df_all_dummies.columns) - {"y", "ds"})


# slicing the input in train test
df_train_all_dummies = df_all_dummies[
    (df_all_dummies["ds"] > "2012") & (df_all_dummies["ds"] < "2015")
]
df_test_all_dummies = df_all_dummies[(df_all_dummies["ds"] > "2015")]

X_train = df_train_all_dummies.loc[:, all_features]
y_train = df_train_all_dummies[["y"]]

X_test = df_test_all_dummies.loc[:, all_features]
y_test = df_test_all_dummies[["y"]]

In [ ]:
# create the pipeline
minmax_all_dummies = MinMaxScaler()
linereg_all_dummies = LinearRegression()
pipeline_multiple_dummies = make_pipeline(minmax_all_dummies, linereg_all_dummies)

pipeline_multiple_dummies.fit(X=X_train, y=y_train)

y_pred_all_dummies = pipeline_multiple_dummies.predict(X=X_test)

mse_all_dummies = mean_squared_error(y_test, y_pred_all_dummies)
mae_all_dummies = mean_absolute_error(y_test, y_pred_all_dummies)
print(f"mean squared error = {mse_all_dummies:.3}")
print(f"mean absolute error = {mae_all_dummies:.3}")

In [ ]:
plt.plot_date(
    x=df_train["ds"],
    y=df_train["y"],
    label="train",
    fmt="-",
)
plt.plot_date(
    x=df_test["ds"],
    y=df_test["y"],
    label="test",
    fmt="-",
)

plt.plot_date(
    x=df_test["ds"],
    y=y_pred_all_dummies,
    label="prediction /w all dummies",
    fmt="-",
)


plt.legend(loc="upper right")
plt.xlabel("date")
plt.ylabel("visits log transformed ")
plt.title(
    f"linear regression applied (mse= {mse_all_dummies:.3}, mae={mae_all_dummies:.3})"
)
if PLOT_FIGS:
    plt.savefig(f"01_linear_forecast/{next(fig_counter)}_all_dummy_predict.jpg")

In [ ]:
# create the pipeline
minmax_all_dummies = MinMaxScaler()
elastic_all_dummies = ElasticNetCV(
    l1_ratio=np.linspace(1e-3, 1, num=100),
    cv=7,
    n_alphas=1_000,
)
pipeline_all_dummies_elastic = make_pipeline(minmax_all_dummies, elastic_all_dummies)

pipeline_all_dummies_elastic.fit(X=X_train, y=y_train.values.ravel())

y_pred_all_dummies = pipeline_all_dummies_elastic.predict(X=X_test)

mse_all_dummies = mean_squared_error(y_test, y_pred_all_dummies)
mae_all_dummies = mean_absolute_error(y_test, y_pred_all_dummies)
print(f"mean squared error = {mse_all_dummies:.3}")
print(f"mean absolute error = {mae_all_dummies:.3}")

In [ ]:
plt.plot_date(
    x=df_train["ds"],
    y=df_train["y"],
    label="train",
    fmt="-",
)
plt.plot_date(
    x=df_test["ds"],
    y=df_test["y"],
    label="test",
    fmt="-",
)

plt.plot_date(
    x=df_test["ds"],
    y=y_pred_all_dummies,
    label="prediction /w all dummies",
    fmt="-",
)


plt.legend(loc="upper right")
plt.xlabel("date")
plt.ylabel("visits log transformed ")
plt.title(
    f"linear regression applied (mse= {mse_all_dummies:.3}, mae={mae_all_dummies:.3})"
)
if PLOT_FIGS:
    plt.savefig(f"01_linear_forecast/{next(fig_counter)}_all_dummy_elastic_predict.jpg")

In [ ]:
df_elastic_coef = pd.DataFrame(
    columns=["coef", "feature"],
    data=np.hstack(
        [
            pipeline_all_dummies_elastic["elasticnetcv"].coef_.reshape(-1, 1),
            X_train.columns.values.reshape(-1, 1),
        ]
    ),
)

In [ ]:
sns.barplot(
    data=df_elastic_coef[
        df_elastic_coef["coef"] > np.percentile(df_elastic_coef["coef"], q=80)
    ].sort_values("coef", ascending=False),
    y="feature",
    x="coef",
    orient="horizontal",
)